In [1]:
import numpy as np

Vanilla neural network does not handle sequential data. In light of this, we have RNN, which is a special type of neural network that is good in modelling sequential data. Examples of sequential data include text, audio, time series, etc. 

Below picture offers a great visualization of the mechanism of RNN under the hood. 

<img src="https://camo.githubusercontent.com/9ecb85b81652e0442a635261463ea3ddae39512bf3d928b0a1f5b8df86ee4ca0/68747470733a2f2f6769746875622e636f6d2f446565704c6561726e696e674454552f30323435362d646565702d6c6561726e696e672d776974682d5079546f7263682f626c6f622f6d61737465722f7374617469635f66696c65732f726e6e2d756e666f6c642e706e673f7261773d31" width="800">


RNN is effectively a loop, where RNN unit processes one unit of the input sequence at a time and then passes the result to the next one. As results are being propagated down the loop, latter RNN units will have memory about the previous units of the sequence. 

In [2]:
# there are different strategies to parameter initialization which would speed up the training, e.g. Xavier, however in this example we only use the good O' random initialization method
def init_params(hidden_size, vocab_size):
    U = np.random.randn(hidden_size, vocab_size) * 0.1 # weights matrix applied on input
    V = np.random.randn(hidden_size, hidden_size) * 0.1
    W = np.random.randn(vocab_size, hidden_size) * 0.1
    bh = np.zeros((hidden_size, 1)) # hidden state bias
    by = np.zeros((vocab_size, 1)) # output bias
    return U, V, W, bh, by


In [3]:
# activation functions
def sigmoid(x, derivative=False): # squish the value between [0,1]
    if not derivative:
        return 1/(1+np.exp(-x))
    else:
        return sigmoid(x)*(1-sigmoid(x))
    
    
def tanh(x, derivative=False): # squish the value between [-1,1]
    if not derivative:
        return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    else:
        return 1 - tanh(x)**2
    
    
def softmax(x): # convert numbers into probabilities
    return np.exp(x) / np.sum(np.exp(x))

        
    

In [4]:
# forward propagation
# the input data, ie. the sequential data, is in the form of an array of vectors
# each vector is passed to the RNN one by one
# in each loop, there are two outputs, the output & the hidden state

def forward_propagation(x, params, hidden_size):
    # unpack params
    U, V, W, bh, by = params
    
    # init outputs, hidden_states, h_prev
    outputs, hidden_states = [], []
    h_prev = np.zeros((hidden_size,1))
    for i in x:
        
        # update current hidden state with new inputs
        h_curr = tanh(np.dot(U, i) + np.dot(V, h_prev) + bh)
        
        # make prediction based on the current hidden states, which combines current input and previous hidden states
        y_hat = softmax(np.dot(W, h_curr) + by)
        
        # save the results
        hidden_states.append(h_curr)
        outputs.append(y_hat)

        # update h_prev for next iteration
        h_prev = h_curr

    return outputs, hidden_states


def cross_entropy_loss(y_hat, y):
    epsilon = 1e-12 # avoid log(0)
    return -np.mean(np.log(y_hat + epsilon) * y)


def backward_propagation(inputs, targets, outputs, hidden_states, params, learning_rate=0.01):
    # unpack parameters
    U, V, W, bh, by = params

    # initialize gradients as zeros
    dU, dV, dW, dbh, dby = np.zeros_like(U), np.zeros_like(
        V), np.zeros_like(W), np.zeros_like(bh), np.zeros_like(by)

    # initialize loss - the total loss will be the sum of loss of each timestep of the input sequence
    loss = 0
    
    # initialize hidden state derivatives
    # need to keep track of hidden state derivatives of each timestep since the dh of a given timestep is the sum of current dh & the dh of the next timestep
    dh_next = np.zeros_like(hidden_states[0])
    
    # compute the gradients from back to start
    for t in reversed(range(len(outputs))):
        # t -> time-step
        
        # compute the loss and add to total loss
        loss += cross_entropy_loss(y_hat=outputs[t], y=targets[t])
        
        # derivative of loss wrt y_hat
        dy = outputs[t].copy()
        # derivation proof: https://cs231n.github.io/neural-networks-case-study/#grad
        # dy[np.argmax(targets[t])] -= 1 -- from Andrej Karpathy's example (https://gist.github.com/karpathy/d4dee566867f8291f086)
        dy = dy - targets[t] # tho less efficient, this line has the same output as the above, but I think this is more intuitive and readable
        # dy shape -> (vocab_size, 1)
        
        # derivative of loss wrt W
        # dW shape -> (vocab_size, hidden_size) -> (vocab_size, 1) * (hidden_size,1).T 
        dW = np.dot(dy, hidden_states[t].T)
        dby += dy
        
        # derivative of loss wrt hidden state
        # dh shape -> (hidden_size, 1)
        dh = np.dot(W.T, dy) + dh_next
        
        # derivative of h_raw wrt U -> dh * dh/d_tanh
        # h_raw -> raw hidden state vector before passing to the tanh activation function
        # dh shape -> (hidden_size, 1)
        # d_tanh shape -> (hidden_size, 1)
        # element-wise multiplication instead of np.dot to maintain the shape of (hidden_size, 1)
        dh_raw = dh * tanh(hidden_states[t], derivative=True)
        
        # derivative of loss wrt U -> dh_raw * dh_raw/dU
        # input shape -> (vocab_size, 1)
        # dU = dh_raw * input.T -> (hidden_size ,1) * (vocab_size, 1).T -> (hidden_size, vocab_size)
        dU += np.dot(dh_raw, inputs[t].T)
        
        # derivative of loss wrt V -> dh * dh_raw * dh_raw/dU
        # dV = dh_raw * hidden_state.T -> (hidden_size ,1) * (hidden_size, 1).T -> (hidden_size, hidden_size)
        dV += np.dot(dh_raw, hidden_states[t].T)
        
        # derivative of loss wrt dh_next
        dh_next = np.dot(V.T, dh_raw)
        
        # derivative of loss wrt hb
        dbh += dh_raw
        
    # for dparam in [dU, dV, dW, dbh, dby]:
    #     np.clip(dparam, -1, 1, out=dparam) # clip to mitigate exploding gradients
        
    # put gradients into a tuple
    gradients = (dU, dV, dW, dbh, dby)
    
    # gradient clipping to avoid exploding gradients
    clip_gradients(gradients)
    
    # update parameters
    updated_params = update_params(gradients, params, learning_rate)
    
    # return gradients and loss (for logging)
    return updated_params, loss


# function to update parameters according to the gradients
def update_params(gradients, params, learning_rate):
    for grad, p in zip(gradients, params):
        p -= grad * learning_rate
    return params

# during training gradients may get too large which makes the model unstable, clipping function to avoid exploding gradients
def clip_gradients(gradients, threshold=1):
    for grad in gradients:
        np.clip(grad, a_min=-threshold, a_max=threshold, out=grad)
    

Below picture, taken from a blog post on Medium, makes a good illustration of the effect of gradient clipping. When the gradients are too large, the parameters take a huge descent and leave the good region -- causing instability to the model.

There are different strategies to gradient clipping:
- Gradient Norm Clipping - scaling the derivatives to have a given vector norm
- **Value Clipping** - change the gradient greater/less than a threshold to that threshold. This is the simplest clipping method which we'll use in this example.

<img src="https://miro.medium.com/max/1400/1*vLFINWklJ0BtYtgzwK223g.png" />

In [5]:
from tqdm import tqdm # to print progress bar


def train(x, y, epochs, hidden_size, vocab_size, learning_rate=0.001, log_step=10):
    # init random params
    params = init_params(hidden_size, vocab_size)
    
    # save loss for plotting
    loss_history = []
    
    for e in tqdm(range(epochs)):
        # init epoch loss
        epoch_loss = []
        
        for i in range(len(x)):
            # forward propagation
            outputs, hidden_states = forward_propagation(x[i], params, hidden_size)
            
            # back propagation and update parameters
            params, loss = backward_propagation(x[i], y[i], outputs, hidden_states, params, learning_rate)

            if np.isnan(loss):
                raise ValueError('Gradients have vanished!')
            
            # save loss
            epoch_loss.append(loss)
            
        # calculate the mean loss of the entire epoch    
        loss_history.append(np.mean(epoch_loss))
        
        # print training results
        if e % log_step ==0:
            print(f"Epoch {e}/{epochs} loss: {loss_history[-1]}")
            
    # return trained parameters for using the model        
    return params
        

In [6]:
def predict(x, trained_params, hidden_size, dictionary, top=5, ):  # predict single instance
    # forward propagation on trained parameters
    outputs, hidden_states = forward_propagation(x, trained_params,hidden_size)
    
    # extract the indices of the top5 probabilities of the output vector
    vocab_ix = outputs[-1].flatten().argsort()[::-1][:top]
    
    # get the corresponding probabilities
    prob = outputs[-1][vocab_ix]
    
    # map indices to the corresponding vocab
    preds = [dictionary.get(i) for i in vocab_ix]
    
    # return the top predicted words and corresponding probabilities 
    return list(zip(preds, prob))
    
        

In [9]:
# prepare fake data
# the data we are generating is a sequence of letters, the first 2 letters are randomized, and the subsequent letters depend on the sum of the index of the preceeding letters 
# if the sum exceeds 25, i.e. the last index of all letters, it goes back to the beggining
# e.g. if the index is 30, the corresponding letter would be at index = 30%26 = 4, i.e. 'e' 
# we can think of it as a variant of the Fibonacci Series ;)

# generate data
SEQUENCE_LEN = 5
def generate_data(n_sample, sequence_len):
    words = list("abcdefghijklmnopqrstuvwxyz")
    inputs = []
    outputs = []
    for i in range(n_sample):
        arr = list(np.random.randint(26, size=2))
        while len(arr) < sequence_len+1:
            ix = sum(arr) if sum(arr) < 26 else sum(arr) % 26
            arr.append(ix)
        seq = [words[i] for i in arr]
        inputs.append(seq[:-1])
        outputs.append(seq[1:])
    return inputs, outputs


# function to encode the data
def encode_data(inputs, outputs):
    words = list("abcdefghijklmnopqrstuvwxyz")

    word_ix = dict((c, i) for i, c in enumerate(words))

    # initialize x,y as zeros
    x = np.zeros(shape=(len(inputs), SEQUENCE_LEN, len(words), 1))
    y = np.zeros(shape=(len(inputs), SEQUENCE_LEN, len(words), 1))

    # loop to update x,y
    for ix, sample in enumerate(inputs):
        for t, word in enumerate(sample):
            x[ix, t, word_ix[word]] = 1
            y[ix, t, word_ix[outputs[ix][t]]] = 1

    return x, y


input_data, output_data = generate_data(n_sample=1000, sequence_len=SEQUENCE_LEN)
x, y = encode_data(input_data, output_data)


In [10]:
import pandas as pd

# let's put into a dataframe to better visualize the data
df = pd.DataFrame({
    'input': input_data,
    'output': output_data
})

df.sample(5)

,input,output
475,"[n, w, j, s, k]","[w, j, s, k, u]"
524,"[s, t, l, w, s]","[t, l, w, s, k]"
261,"[w, l, h, o, c]","[l, h, o, c, e]"
606,"[b, p, q, g, m]","[p, q, g, m, y]"
321,"[w, w, s, k, u]","[w, s, k, u, o]"


In [11]:
# define the hyperparameters
epochs = 200
hidden_size = 50
vocab_size = 26
learning_rate = 0.0001 # keeping the lr low is crucial to prevent gradient from blowing up and throw 'nan' loss
log_step = epochs/10

# train!
params = train(x, y, epochs, hidden_size, vocab_size, learning_rate, log_step)


  0%|          | 1/200 [00:00<00:45,  4.40it/s]

Epoch 0/200 loss: 0.6266416735668658


 10%|█         | 21/200 [00:04<00:39,  4.52it/s]

Epoch 20/200 loss: 0.5744540348321556


 20%|██        | 41/200 [00:09<00:35,  4.50it/s]

...

 80%|████████  | 161/200 [00:36<00:08,  4.44it/s]

Epoch 160/200 loss: 0.5026572926760171


 90%|█████████ | 181/200 [00:40<00:04,  4.38it/s]

Epoch 180/200 loss: 0.49423312384303814


100%|██████████| 200/200 [00:44<00:00,  4.46it/s]


In [13]:
# test out the trained model

# select a random sample from the data
rand = np.random.randint(1000)
print("input data: ",input_data[rand])

# test input sample
x_test = x[rand]

# dictionary for mapping ix to word
ix_word = dict((i, c) for i, c in enumerate(list("abcdefghijklmnopqrstuvwxyz")))

# predict
predictions = predict(x_test, params, hidden_size=50, dictionary=ix_word, top=5)
print('predictions: ', predictions)
print('truth: ', output_data[rand][-1])


input data:  ['r', 'r', 'i', 'q', 'g']
predictions:  [('m', array([0.34766399])), ('g', array([0.12381405])), ('e', array([0.11245404])), ('i', array([0.06922135])), ('s', array([0.04318737]))]
truth:  m


Not bad! The model is able to pick up the pattern in just 200 training epochs.